### Connect

In [69]:
%env CYPHER=/usr/bin/cypher-shell
%env HOST=uri
%env USER=neo4j
%env PW=password

env: CYPHER=/usr/bin/cypher-shell
env: HOST=neo4j+s://3b4b1cf9.databases.neo4j.io
env: USER=neo4j
env: PW=emJUcUumf9GdTGQz37htpHYX_id5lmT9b23Ki8EMqR0


In [70]:
CONNECT="$CYPHER -a $HOST -u $USER -p $PW"


In [71]:
from IPython.core.magic import register_cell_magic
import shlex

@register_cell_magic('neo')
def neo(line, cell):
    !{CONNECT} {shlex.quote(cell)}

In [22]:
!{CONNECT} "show users"

+------------------------------------------------------------------+
| user    | roles      | passwordChangeRequired | suspended | home |
+------------------------------------------------------------------+
| "neo4j" | ["PUBLIC"] | FALSE                  | FALSE     | NULL |
+------------------------------------------------------------------+

1 row
ready to start consuming query after 10 ms, results consumed after another 72 ms


In [9]:
!{CONNECT} "MATCH (n) RETURN n LIMIT 5"

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| n                                                                                                                                                                                                                                                                                                         |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| (:Recipe {preparationTime: 900, name: "Tomato & mozzarella couscous salad", description: "Ke

### Begin Project 7 Work

#### Queries

Note: Helps to find relationships on Neo4j: MATCH (n:Recipe)-[r]-(nn:Collection) RETURN distinct n.name, type(r), nn.name \
need to ask how to  Return properties of relationships is they're all empty

Q1 Returns the unique node labels and the number of nodes for each unique node label

In [10]:
!{CONNECT} "MATCH (n) RETURN distinct labels(n), count(*)"

+---------------------------+
| labels(n)      | count(*) |
+---------------------------+
| ["Recipe"]     | 11634    |
| ["Author"]     | 303      |
| ["Ingredient"] | 3077     |
| ["DietType"]   | 12       |
| ["Collection"] | 1049     |
+---------------------------+

5 rows
ready to start consuming query after 3 ms, results consumed after another 16 ms


Q2 Returns the unique relationship types and the number of relationships for each

In [13]:
!{CONNECT} "MATCH (n)-[r]-(m) RETURN distinct type(r), count(*)"

+----------------------------------+
| type(r)               | count(*) |
+----------------------------------+
| "WROTE"               | 23268    |
| "COLLECTION"          | 64480    |
| "DIET_TYPE"           | 29716    |
| "CONTAINS_INGREDIENT" | 212296   |
+----------------------------------+

4 rows
ready to start consuming query after 37 ms, results consumed after another 86 ms


Q3 Returns name, skill level of recipes containing tomato or lemon, and which ingredient is in the recipe. It is sorted by recipe name ascending \
(or1, node prop, sort a)

In [11]:
!{CONNECT} "match (n:Recipe)-[r]-(nn:Ingredient) \
where nn.name='tomato' or nn.name='lemon' \
return n.name as recipe_name, n.skillLevel as level, nn.name as ingredient \
order by recipe_name"

+-----------------------------------------------------------------------------------------------------------------+
| recipe_name                                                                        | level         | ingredient |
+-----------------------------------------------------------------------------------------------------------------+
| "10-minute mincemeat"                                                              | "Easy"        | "lemon"    |
| "10-minute tuna-bean toasts"                                                       | "Easy"        | "tomato"   |
| "30 minute chicken & tarragon pasta"                                               | "Easy"        | "lemon"    |
| "4 creamy dips "                                                                   | "Easy"        | "lemon"    |
| "5-a-day burger"                                                                   | "Easy"        | "lemon"    |
| "5-a-day chicken with kale & pistachio pesto"                         

Q4 Returns recipe name and prep time that are in the collections, Challenging cake and Chocolate cake, sorted by preparation time descending \
(and1, node prop, sort d)

In [16]:
!{CONNECT} "match (n:Recipe)-[r]-(c:Collection) \
where c.name='Chocolate cake' with collect(n.name) as recipes \
match (nn:Recipe)-[rr]-(cc:Collection) \
where cc.name='Challenging cake'and nn.name in recipes \
return nn.name, nn.preparationTime \
order by nn.preparationTime desc"

+-----------------------------------------------------------+
| nn.name                              | nn.preparationTime |
+-----------------------------------------------------------+
| "Malted chocolate drip cake"         | 7200               |
| "Hazelnut latte cake"                | 5400               |
| "Gravity-defying sweetie cake"       | 3600               |
| "Cookies & cream party cake"         | 3600               |
| "Chocolate & caramel ombre cake"     | 3600               |
| "Chocolate meringue Mont Blanc cake" | 3300               |
| "Orange & white chocolate sponge"    | 1800               |
| "White & dark chocolate cake"        | 1800               |
+-----------------------------------------------------------+

8 rows
ready to start consuming query after 888 ms, results consumed after another 71 ms


Q5 Returns 10 authors who have written recipes that do not contain tomatoes and are in the diet types, Low-salt and Healthy \
(and2, neg1, three rel1)

In [87]:
!{CONNECT} "match (n:Author)-[r]-(nn:Recipe) \
where not nn.name contains 'tomato' \
match (nnn:Recipe)-[rr]-(dd:DietType) \
where dd.name = 'Low-salt' \
match (nnnn:Recipe)-[rrr]-(ddd:DietType) \
where ddd.name = 'Healthy' \
return distinct n.name \
limit 10"

+------------------+
| n.name           |
+------------------+
| "Tony Tobin"     |
| "Jane Hornby"    |
| "Sarah Cook"     |
| "Good Food"      |
| "Jennifer Joyce" |
| "Silvana Franco" |
| "Lucy Netherton" |
| "James Martin"   |
| "Mary Cadogan"   |
| "John Torode"    |
+------------------+

10 rows
ready to start consuming query after 833 ms, results consumed after another 40654 ms


Q6 Returns 10 recipes that contain couscous from the Healthy collection and were written by an author with 'a' in their name \
(three rel2)

In [91]:
!{CONNECT} "match (nn:Ingredient)-[:CONTAINS_INGREDIENT]-(nnn:Recipe) \
where nn.name='couscous' \
with collect(nnn.name) as couscous_recipes \
match (n:Recipe)-[:COLLECTION]-(c:Collection) \
where c.name = 'Healthy' and not n.name in couscous_recipes \
match (a:Author)-[:WROTE]-(nn:Recipe) \
where a.name contains 'a' \
return distinct n.name\
limit 10"

+-----------------------------------------------------------+
| n.name                                                    |
+-----------------------------------------------------------+
| "Mushroom & potato soup"                                  |
| "One-pan egg & veg brunch"                                |
| "Spicy chicken & avocado wraps"                           |
| "Haddock with cannellini beans & artichokes"              |
| "Roast aubergine with goulash sauce & sweet potato fries" |
| "Marinated beetroot with grilled goat's cheese"           |
| "Egg & rocket pizzas"                                     |
| "Minted courgette salad"                                  |
| "Greek island salad with chicken & avocado"               |
| "Mexican penne with avocado"                              |
+-----------------------------------------------------------+

10 rows
ready to start consuming query after 331 ms, results consumed after another 9 ms


Q7 Returns 10 names of authors who have written in diet types, Low-sugar or Vegan \
(or2)

In [3]:
!{CONNECT} "match (a:Author)-[r]-(n:Recipe) \
match (nn:Recipe)-[rr]-(d:DietType) \
where d.name='Low-sugar' or d.name='Vegan' \
return distinct a.name\
limit 10"

+------------------+
| a.name           |
+------------------+
| "Tony Tobin"     |
| "Jane Hornby"    |
| "Sarah Cook"     |
| "Good Food"      |
| "Jennifer Joyce" |
| "Silvana Franco" |
| "Lucy Netherton" |
| "James Martin"   |
| "Mary Cadogan"   |
| "John Torode"    |
+------------------+

10 rows
ready to start consuming query after 24 ms, results consumed after another 1825 ms


Q8 Returns 10 recipes that are not in collection Healthy  
(neg2)

In [5]:
!{CONNECT} "match (n:Recipe)-[r]-(c:Collection) \
where not c.name = 'Healthy' \
return distinct n.name\
limit 10"

+---------------------------------------------------------------+
| n.name                                                        |
+---------------------------------------------------------------+
| "Nectarine & pistachio crunch layers"                         |
| "Herby feta & nectarine salad with lemon poppy seed dressing" |
| "Barley couscous & prawn tabbouleh"                           |
| "Nectarine & raspberry gratin"                                |
| "Summer fruit quinoa salad"                                   |
| "Roasted stone fruits"                                        |
| "Summer greens & nectarines"                                  |
| "Spice roasted fruits with honey & orange sauce"              |
| "Giant peach, nectarine & apricot meringue tart"              |
| "Sunshine tiramisu"                                           |
+---------------------------------------------------------------+

10 rows
ready to start consuming query after 70 ms, results consumed after 

Q9 Returns 10 relationship IDS of recipes in Challenge cake  
(rel prop)

In [89]:
!{CONNECT} "match (n:Recipe)-[r]-(c:Collection) \
where c.name='Chocolate cake' \
return ID(r) \
limit 10"

+-------+
| ID(r) |
+-------+
| 17137 |
| 23402 |
| 23778 |
| 13710 |
| 25800 |
| 41098 |
| 42422 |
| 39009 |
| 13679 |
| 37508 |
+-------+

10 rows
ready to start consuming query after 209 ms, results consumed after another 9 ms


Q10 Returns authors who have written recipes that contain basil

In [72]:
!{CONNECT} "match (n:Author)-[r]-(nn:Recipe) \
match (nnn:Recipe)-[rr]-(dd:Ingredient) \
where dd.name contains 'basil' \
return distinct n.name \
limit 10"

+------------------+
| n.name           |
+------------------+
| "Tony Tobin"     |
| "Jane Hornby"    |
| "Sarah Cook"     |
| "Good Food"      |
| "Jennifer Joyce" |
| "Silvana Franco" |
| "Lucy Netherton" |
| "James Martin"   |
| "Mary Cadogan"   |
| "John Torode"    |
+------------------+

10 rows
ready to start consuming query after 22 ms, results consumed after another 2296 ms


### Begin Extra Credit Work

M1 Created a new node for each existing node label with one property name

In [125]:
%%neo
CREATE (:Author {name:"Tony Wang"}), (:Recipe {name:"Fried Rice"}), (:Ingredient {name:"Eggs and Rice"}), (:DietType {name:"KindOfHealthy"}), (:Collection {name: "Chinese Food"});

0 rows
ready to start consuming query after 133 ms, results consumed after another 0 ms
Added 5 nodes, Set 5 properties, Added 5 labels


M2 Created a new relationship for each existing relationship type

In [126]:
%%neo
MATCH (a:Author {name: "Tony Wang"})
MATCH (r:Recipe {name:"Fried Rice"})
CREATE (a)-[:WROTE]->(r);

MATCH (rr:Recipe {name:"Fried Rice"})
MATCH (i:Ingredient {name:"Eggs and Rice"})
CREATE (rr)-[:CONTAINS_INGREDIENT]->(i);

MATCH (rrr:Recipe {name:"Fried Rice"})
MATCH (d:DietType {name:"KindOfHealthy"})
CREATE (rrr)-[:DIET_TYPE]->(d);

MATCH (rrrr:Recipe {name:"Fried Rice"})
MATCH (c:Collection {name: "Chinese Food"})
CREATE (rrrr)-[:COLLECTION]->(c);

0 rows
ready to start consuming query after 134 ms, results consumed after another 0 ms
Created 4 relationships
0 rows
ready to start consuming query after 115 ms, results consumed after another 0 ms
Created 16 relationships
0 rows
ready to start consuming query after 98 ms, results consumed after another 0 ms
Created 16 relationships
0 rows
ready to start consuming query after 103 ms, results consumed after another 0 ms
Created 16 relationships


M3 Updates Name of Inserted Author Node to "Tony Wang"

In [131]:
%%neo
MATCH (a:Author {name: "Tony Wang"})
set a.name = "Tony J Wang"
return a.name;

+---------------+
| a.name        |
+---------------+
| "Tony J Wang" |
+---------------+

1 row
ready to start consuming query after 16 ms, results consumed after another 4 ms
Set 1 properties


M4 Adds new node label :recipe to inserted author node

In [132]:
%%neo
MATCH (a:Author {name: "Tony J Wang"})
SET a:Recipe
Return a.name, labels(a) AS labels;

+--------------------------------------+
| a.name        | labels               |
+--------------------------------------+
| "Tony J Wang" | ["Recipe", "Author"] |
+--------------------------------------+

1 row
ready to start consuming query after 74 ms, results consumed after another 3 ms
Added 1 labels


M5 Update the properties of one or more relationships

In [133]:
%%neo
MATCH (a:Author {name:"Tony J Wang"})-[r]-(n:Recipe {name:"Fried Rice"})
SET r.P = "something"

0 rows
ready to start consuming query after 77 ms, results consumed after another 0 ms
Set 4 properties


M6 Create a unique constraint on node property name

In [137]:
%%neo
DROP CONSTRAINT constraint_8ba090b IF EXISTS

0 rows
ready to start consuming query after 27 ms, results consumed after another 0 ms
Removed 1 constraints


In [138]:
%%neo
CREATE CONSTRAINT FOR (a:Author) REQUIRE a.name IS UNIQUE

0 rows
ready to start consuming query after 75 ms, results consumed after another 0 ms
Added 1 constraints


M7 Update the relationship type of at least one relationship

In [ ]:
%%neo
MATCH (a:Author {name:"Tony J Wang"})-[r:WROTE]->(n:Recipe {name:"Fried Rice"})
CREATE (a)-[r2:COLLECTION]->(n)
// copy properties, if necessary
SET r2 = r
WITH r
DELETE r

0 rows
ready to start consuming query after 169 ms, results consumed after another 0 ms
Created 3 relationships, Deleted 3 relationships, Set 3 properties


M8 Create a unique constraint on a relationship property

In [124]:
%%neo
CREATE CONSTRAINT year_wrote
FOR ()-[r:COLLECTION]-() REQUIRE r.date IS UNIQUE

0 rows
ready to start consuming query after 106 ms, results consumed after another 0 ms
Added 1 constraints


M9 Deletes relationship between inserted author and recipe node based on some criteria

In [129]:
%%neo
MATCH (a:Author {name: "Tony J Wang"})
MATCH (n:Recipe {name:"Fried Rice"})
MATCH (a)-[r:COLLECTION]->(n)
DELETE r

0 rows
ready to start consuming query after 97 ms, results consumed after another 0 ms
Deleted 3 relationships


M10 Delete at least one node based on some criteria

In [130]:
%%neo
MATCH (a:Author) 
WHERE a.name="Tony J Wang" 
DELETE a

0 rows
ready to start consuming query after 15 ms, results consumed after another 0 ms
Deleted 1 nodes
